# 交叉随机基准测试获取表面码基本门的噪声模型

在Surface code电路中，我们通过只包含CNOT门和H门，除此之外可能还要I门表示空闲。

因为我们考虑利用交叉随机基准测试的方法，来获取对应的CNOT门、H门作用之后可能引入的去极化噪声到底是多少，同时尝试验证其准确性。

In [1]:
import numpy as np

from qiskit.providers.fake_provider import FakeSingaporeV2
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

在qiskit考虑的门噪声模型中，是由一个depolarizing noise和一个thermal relaxation noise组合而成。其中thermal relaxation noise是退相干噪声，我们这里只考虑depolarizing noise，即去极化噪声。

IRB方法是获取去极化参数，depolarizing noise可以较为精准准确获取，但获取thermal relaxation noise部分会将其转换为depolarizing noise，从而存在误差。

获取对应噪声模型以及其含噪模拟后端，其中前者包含退相干噪声，后者不包含。

In [2]:
backend = AerSimulator.from_backend(FakeSingaporeV2())
backend_noise_model = NoiseModel.from_backend(backend)

backend_noise_model_no_thermal_relaxation = NoiseModel.from_backend(backend,thermal_relaxation=False)
backend_no_thermal_relaxation = AerSimulator(noise_model = backend_noise_model_no_thermal_relaxation, n_qubits = backend.num_qubits, coupling_map = backend.coupling_map, basis_gates = backend._basis_gates())

设置交叉随机基准测试的参数。

In [3]:
lengths = np.arange(1, 121+1, 30)
num_samples = 10
seed = 1010

通过交叉随机基准测试，获取在对应噪声模型上运行，CNOT门和H门作用后引入的去极化噪声参数为多少。
普通计算机，在上述噪声模型下，大约运行4分多钟

In [4]:
from irb_for_surface_code_gate import interleaved_rb_QEC, get_surface_code_noise_model

h_gate_error, id_gate_error, cx_gate_error, unique_pairs = interleaved_rb_QEC(backend, lengths, num_samples, seed)

h_gate_error_no_thermal_relaxation, id_gate_error_no_thermal_relaxation, cx_gate_error_no_thermal_relaxation, unique_pairs = interleaved_rb_QEC(backend_no_thermal_relaxation, lengths, num_samples, seed)

构建CNOT、H、I门的噪声模型。

In [5]:
sim_noise, noise_model = get_surface_code_noise_model(h_gate_error, id_gate_error, cx_gate_error, backend, backend_noise_model)

sim_noise_no_thermal_relaxation, no_thermal_relaxation_noise_model = get_surface_code_noise_model(h_gate_error_no_thermal_relaxation, id_gate_error_no_thermal_relaxation, cx_gate_error_no_thermal_relaxation, backend_no_thermal_relaxation, backend_noise_model_no_thermal_relaxation)

对比噪声模型的I门和通过IRB学习到的噪声模型I门的过程保真度

In [6]:
from irb_for_surface_code_gate import i_gate_fidelity
i_gate_fidelity(id_gate_error, id_gate_error_no_thermal_relaxation, backend_noise_model, backend_noise_model_no_thermal_relaxation)

考虑thermal_relaxiaiton噪声, 实际和学习到的噪声模型之间的噪声通道过程保真度: 0.9997780019682114
不考虑thermal_relaxiaiton噪声, 实际和学习到的噪声模型之间的噪声通道过程保真度: 0.999974779254305


我们可以看到，不考虑thermal_relaxiaiton比考虑thermal_relaxiaiton，过程保真度高了一个数据级。即使考虑thermal_relaxiaiton，学习到的噪声模型与实际噪声模型之间的过程保真度也相当高。

考虑CNOT门的结果如下所示：

In [7]:
# 考虑CNOT门
from irb_for_surface_code_gate import cnot_gate_process_fidelity

cnot_gate_process_fidelity(cx_gate_error, cx_gate_error_no_thermal_relaxation, backend_noise_model, backend_noise_model_no_thermal_relaxation)

ImportError: cannot import name 'cnot_gate_process_fidelity' from 'irb_for_surface_code_gate' (c:\Users\86178\Desktop\Senior\Quantum-research\LINKEQ\quantum-noise-learning\quantum_noise_learning\interleaved_randomized_benchmarking\irb_for_surface_code_gate.py)

由于本身不支持H门，因为我们通过hellinger_fidelity和hellinger_distance来估计。

考虑退相干噪声：

In [ ]:
from irb_for_surface_code_gate import h_gate_hellinger_fidelity

h_gate_hellinger_fidelity(sim_noise, backend)

学习噪声模型结果:{'0': 18769, '1': 1711}, 实际噪声模型结果:{'0': 19722, '1': 758}
hellinger距离:0.0704227908009819, hellinger保真度:0.9901058564170847


不考虑退相干噪声：

In [ ]:
h_gate_hellinger_fidelity(sim_noise_no_thermal_relaxation, backend_no_thermal_relaxation)

学习噪声模型结果:{'0': 19587, '1': 893}, 实际噪声模型结果:{'0': 19701, '1': 779}
hellinger距离:0.009951401955855445, hellinger保真度:0.9998019490052463
